In [1]:
import pandas as pd
df1 = pd.read_csv("bollywood.csv")
df2 = pd.read_csv("bollywood - 2.csv")

In [2]:
df1.head()

,No,RelDate,MovieName,ReleaseTime,Genre,Budget,BoxOfficeCollection,YoutubeViews,YoutubeLikes,YoutubeDislikes,YoutubeLikeDislikes
0,1,Apr-18-2014,2 States,3,RomanceGenre,36Cr,104,8576361,26622.0,2527.0,29149
1,2,Jan-4-2013,Table No. 21,4,Thriller,Cr10,12,1087320,1129.0,137.0,1266
2,3,Jul-18-2014,Amit Sahni Ki List,4,ComedyGenre,10Cr,4,572336,586.0,54.0,640
3,4,Jan-4-2013,Rajdhani Express,4,DramaGenre,7Cr,383,42626,86.0,19.0,105
4,5,Jul-4-2014,Bobby Jasoos,44,ComedyGenre,18Cr,10.8,3113427,4512.0,1224.0,5736


In [3]:
df2.head()

,Movie_ID,Release_Date,Movie_Name,Release_Time,Genre,BudgetCr,BoxOfficeCollectionCr,Youtube_Views,Youtube_Likes,Youtube_Dislikes
0,51,4-Jul-14,Lekar Hum Deewana Dil,N,Romance,16,2.50,3998331,2307,497
1,52,22-Mar-13,Aatma,N,Thriller,8,8.78,841891,997,306
2,53,4-Apr-14,Main Tera Hero,LW,Comedy,40,55.00,4903941,12657,2139
3,54,22-Mar-13,Sona Spa,N,Thriller,10,0.24,79430,61,112
4,55,29-Mar-13,Himmatwala,N,Action,50,100.00,2501277,1,1


#### Q1. Identify and rectify the eight prominent data quality issues present in the given datasets.
   * 0.5 * 8 = 4M

1. Column names between data-sets differ | Rename columns to a common name between the sets
* df2 has a missing column: YoutubeLikeDislikes | Create this missing column in df2
* Impute missing values in df1
* Remove cr from budget and box office collection column
* Correct month name spellings in release date
* Clean Genre column to have all unique values
* Correct error in release time column
* Assign appropriate datatypes to columns
* Check for duplicate movie names

1. Column names between data-sets differ 
    * Rename columns to a common name between the sets otherwise we'll have multiple columns with same data but different names after merging

In [4]:
df1.rename(columns={'No':'Movie_ID',
                         'RelDate':'Release_Date',
                         'MovieName':'Movie_Name',
                         'ReleaseTime':'Release_Time',
                         'YoutubeViews':'Youtube_Views',
                         'YoutubeLikes':'Youtube_Likes',
                         'YoutubeDislikes':'Youtube_Dislikes',
                         'YoutubeLikeDislikes':'Youtube_LikeDislikes'},inplace=True)

df2.rename(columns={     'BudgetCr':'Budget',
                         'BoxOfficeCollectionCr':'BoxOfficeCollection'},inplace=True)


2. df2 has a missing column: Youtube_LikeDislikes 
    * Create this missing column in df2 otherwise the 2 datasets would be of different size and they would not merge

In [5]:
df2["Youtube_LikeDislikes"] =  df2.Youtube_Likes + df2.Youtube_Dislikes

3. Impute missing values in df1
    * We cannot train a machine learning model with data having missing values
    * Depending on the amount of missing values, We either impute them or drop them

In [6]:
df1.isnull().sum()

Movie_ID                0
Release_Date            0
Movie_Name              0
Release_Time            0
Genre                   0
Budget                  0
BoxOfficeCollection     0
Youtube_Views           0
Youtube_Likes           1
Youtube_Dislikes        3
Youtube_LikeDislikes    0
dtype: int64

* Here we are imputing missing values in likes and dislikes by calculating them from other columns
    - YouTube_Likes + Youtube_Dislikes = YouTube_LikesDislikes
    - You can also impute them with other appropriate methods such as mean

In [7]:
df1.loc[df1.Youtube_Likes.isna(),"Youtube_Likes"] = df1.loc[df1.Youtube_Likes.isna(),"Youtube_LikeDislikes"] - df1.loc[df1.Youtube_Likes.isna(),"Youtube_Dislikes"]
df1.loc[df1.Youtube_Dislikes.isna(),"Youtube_Dislikes"] = df1.loc[df1.Youtube_Dislikes.isna(),"Youtube_LikeDislikes"] - df1.loc[df1.Youtube_Dislikes.isna(),"Youtube_Likes"]

In [8]:
df2.isnull().sum()

Movie_ID                0
Release_Date            0
Movie_Name              0
Release_Time            0
Genre                   0
Budget                  0
BoxOfficeCollection     0
Youtube_Views           0
Youtube_Likes           0
Youtube_Dislikes        0
Youtube_LikeDislikes    0
dtype: int64

4. Remove cr from budget and box office collection column
    * Budget and box office collection are numeric columns, Hence the value cr should be removed before converting it to integer


In [9]:
df1.BoxOfficeCollection.value_counts()

6         2
383       2
65        1
Cr0.09    1
2.5       1
69        1
103       1
4         1
104       1
14.3      1
40        1
1.8       1
35        1
76.7      1
27        1
162       1
11        1
10        1
16        1
27.16     1
25        1
39.4      1
22        1
63.7      1
21.5      1
10.25     1
32.7      1
10.8      1
130       1
62.2      1
5.82      1
24.6      1
377       1
32        1
27.25     1
36        1
58.9      1
129       1
12        1
Cr0.5     1
44        1
2         1
12.6      1
163       1
105.5     1
1.5       1
3.65      1
111       1
Cr0.01    1
Name: BoxOfficeCollection, dtype: int64

In [10]:
df1.Budget.value_counts()

12Cr     5
10Cr     4
7Cr      3
30Cr     3
15Cr     2
5Cr      2
Cr10     2
36Cr     2
35Cr     2
50Cr     2
4Cr      2
75Cr     1
2Cr      1
Cr60     1
120Cr    1
Cr5      1
33Cr     1
48Cr     1
24Cr     1
18Cr     1
11Cr     1
42Cr     1
8Cr      1
100Cr    1
Cr30     1
31Cr     1
125Cr    1
20Cr     1
21Cr     1
60Cr     1
27Cr     1
28Cr     1
16Cr     1
Name: Budget, dtype: int64

In [11]:
df1.BoxOfficeCollection = df1.BoxOfficeCollection.str.replace('Cr','')
df1.Budget = df1.Budget.str.replace('Cr','')


In [12]:
df1.BoxOfficeCollection.value_counts()

6        2
383      2
65       1
130      1
2.5      1
69       1
0.01     1
4        1
104      1
14.3     1
40       1
1.8      1
35       1
76.7     1
63.7     1
0.09     1
0.5      1
11       1
10       1
16       1
27.16    1
25       1
39.4     1
162      1
27       1
10.25    1
21.5     1
10.8     1
103      1
62.2     1
5.82     1
24.6     1
377      1
32       1
27.25    1
36       1
58.9     1
129      1
12       1
44       1
2        1
12.6     1
163      1
105.5    1
1.5      1
3.65     1
111      1
32.7     1
22       1
Name: BoxOfficeCollection, dtype: int64

In [13]:
df1.Budget.value_counts()

10     6
12     5
30     4
7      3
5      3
60     2
4      2
35     2
36     2
15     2
50     2
120    1
75     1
16     1
20     1
42     1
21     1
2      1
33     1
8      1
27     1
100    1
24     1
48     1
18     1
11     1
125    1
31     1
28     1
Name: Budget, dtype: int64

5. Correct month name spellings in release date.


In [14]:
df1['Release_Date'] = df1['Release_Date'].str.replace(r'Sqp', 'Sep')
df1['Release_Date'] = df1['Release_Date'].str.replace(r'Jum', 'Jun')
df1['Release_Date'] = df1['Release_Date'].str.replace(r'juu', 'Jun')
df1['Release_Date'] = df1['Release_Date'].str.replace(r'Feb-21-204', 'Feb-21-2014')

6. Clean Genre column to have all unique values

In [15]:
df1.Genre.unique()

array(['RomanceGenre', 'Thriller ', 'ComedyGenre', ' DramaGenre',
       'ActionGenre', 'Action ', 'Thriller', ' Drama '], dtype=object)

In [16]:
df1['Genre'] = df1['Genre'].str.replace(r'Genre', '')
df1['Genre'] = df1['Genre'].str.replace(r'Thriller ', 'Thriller')
df1['Genre'] = df1['Genre'].str.replace(r' Drama', 'Drama')
df1['Genre'] = df1['Genre'].str.replace(r'Drama ', 'Drama')
df1['Genre'] = df1['Genre'].str.replace(r'Action ', 'Action')

In [17]:
df1.Genre.unique()

array(['Romance', 'Thriller', 'Comedy', 'Drama', 'Action'], dtype=object)

In [18]:
df2.Genre.unique()

array(['Romance', 'Thriller', 'Comedy', 'Action', ' Drama '], dtype=object)

In [19]:
df2['Genre'] = df2['Genre'].str.replace(r'Drama ', 'Drama')

In [20]:
df2.Genre.unique()

array(['Romance', 'Thriller', 'Comedy', 'Action', ' Drama'], dtype=object)

7. Correct error in release time column

In [21]:
df1.Release_Time.value_counts()

4     33
2     10
3      4
1      3
44     1
Name: Release_Time, dtype: int64

* 44 only appears once. Which suggest it is a mistake and is equivalent to 4 

In [22]:
df1.Release_Time.unique()

array([ 3,  4, 44,  2,  1])

In [23]:
df1['Release_Time'] = df1['Release_Time'].replace(44,4)

In [24]:
df1.Release_Time.unique()

array([3, 4, 2, 1])

In [25]:
df2.Release_Time.unique()

array(['N', 'LW', 'HS', 'FS'], dtype=object)

* Release time of movie:
    - Long Weekend (3), 
    - Festive Season (1), 
    - Holiday Season (2), 
    - Normal (4)

In [26]:
df2['Release_Time'] = df2['Release_Time'].replace(to_replace='N',value=4)
df2['Release_Time'] = df2['Release_Time'].replace(to_replace='FS',value=1)
df2['Release_Time'] = df2['Release_Time'].replace(to_replace='LW',value=3)
df2['Release_Time'] = df2['Release_Time'].replace(to_replace='HS',value=2)

In [27]:
df2.Release_Time.unique()

array([4, 3, 2, 1])

8. Assign appropriate datatypes to columns

In [28]:
df = pd.concat([df1, df2])

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 0 to 98
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Movie_ID              150 non-null    int64  
 1   Release_Date          150 non-null    object 
 2   Movie_Name            150 non-null    object 
 3   Release_Time          150 non-null    int64  
 4   Genre                 150 non-null    object 
 5   Budget                150 non-null    object 
 6   BoxOfficeCollection   150 non-null    object 
 7   Youtube_Views         150 non-null    int64  
 8   Youtube_Likes         150 non-null    float64
 9   Youtube_Dislikes      150 non-null    float64
 10  Youtube_LikeDislikes  150 non-null    int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 14.1+ KB


In [30]:
df = df.astype({'Budget': float, 'BoxOfficeCollection': float,'Youtube_Views' : float,'Release_Time'  : int, 
                'Genre':'string','Movie_Name':'string'})

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 0 to 98
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Movie_ID              150 non-null    int64  
 1   Release_Date          150 non-null    object 
 2   Movie_Name            150 non-null    string 
 3   Release_Time          150 non-null    int64  
 4   Genre                 150 non-null    string 
 5   Budget                150 non-null    float64
 6   BoxOfficeCollection   150 non-null    float64
 7   Youtube_Views         150 non-null    float64
 8   Youtube_Likes         150 non-null    float64
 9   Youtube_Dislikes      150 non-null    float64
 10  Youtube_LikeDislikes  150 non-null    int64  
dtypes: float64(5), int64(3), object(1), string(2)
memory usage: 14.1+ KB


9. Check for duplicate movie names.
    * We do not want duplicate values in our data which makes our data biased

In [32]:
print(df.shape)
df['Movie_Name'].value_counts()


(150, 11)


Lekar Hum Deewana Dil      2
Go Goa Gone                1
Mere Dad Ki Maruti         1
Baby                       1
The Shaukeens              1
                          ..
Ramaiya Vastavaiya         1
Yeh Jawaani Hai Deewani    1
Sona Spa                   1
Holiday                    1
Bang Bang                  1
Name: Movie_Name, Length: 149, dtype: Int64

In [33]:
df.drop_duplicates("Movie_Name",inplace=True)
df.reset_index(drop=True,inplace=True)

In [34]:
print(df.shape)

df['Movie_Name'].value_counts()


(149, 11)


Bajatey Raho               1
Kill Dil                   1
Dishkiyaoon                1
Mere Dad Ki Maruti         1
Baby                       1
                          ..
O Teri                     1
Ramaiya Vastavaiya         1
Yeh Jawaani Hai Deewani    1
Bewakoofiyan               1
Commando-A One Man Army    1
Name: Movie_Name, Length: 149, dtype: Int64

#### Q2. How many records are present in the dataset? Print the metadata information of dataset. 
* 0.5M

In [35]:
records = df.shape[0] 
print("The number of records present in the dataset are: ",records)

The number of records present in the dataset are:  149


In [36]:
print("METADATA INFORMATION\n")
print("-------------------------------------------------")
df.info()

METADATA INFORMATION

-------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Movie_ID              149 non-null    int64  
 1   Release_Date          149 non-null    object 
 2   Movie_Name            149 non-null    string 
 3   Release_Time          149 non-null    int64  
 4   Genre                 149 non-null    string 
 5   Budget                149 non-null    float64
 6   BoxOfficeCollection   149 non-null    float64
 7   Youtube_Views         149 non-null    float64
 8   Youtube_Likes         149 non-null    float64
 9   Youtube_Dislikes      149 non-null    float64
 10  Youtube_LikeDislikes  149 non-null    int64  
dtypes: float64(5), int64(3), object(1), string(2)
memory usage: 12.9+ KB


#### Q3. How many movies in each genre got released in different release times? 
* 0.5M

* grouping by genre column and finding the value counts for different release time.

In [37]:
df.groupby(['Genre'])['Release_Time'].value_counts()

Genre     Release_Time
 Drama    4               18
          2                3
          1                2
Action    4               15
          1                3
          2                3
          3                3
Comedy    4               23
          2                5
          3                5
          1                3
Drama     4                6
          2                3
          1                2
          3                1
Romance   4               15
          3                4
          1                3
          2                3
Thriller  4               22
          1                4
          3                2
          2                1
Name: Release_Time, dtype: int64

#### Q4. Which month of the year, maximum movie releases are seen?
* 1M

* Using feature engineering to create new columns: Year & Month

In [54]:
df['Year'] = pd.DatetimeIndex(df['Release_Date']).year
df['Month'] = pd.DatetimeIndex(df['Release_Date']).month_name()

* Grouping by month for all the years and finding movie counts

In [53]:
df.groupby(['Month'])['Movie_ID'].count().sort_values(ascending=False).reset_index().iloc[0]

Month       January
Movie_ID         20
Name: 0, dtype: object

# OR

* Grouping by each year and it's respective month and finding movie counts

In [52]:
df.groupby(['Year','Month'])['Movie_ID'].count().sort_values(ascending=False).reset_index().iloc[0]

Year        2014
Month        May
Movie_ID      12
Name: 0, dtype: object

#### Q5. Which month of the year typically sees most releases of high budgeted movies, that is, movies with budget of 25 crore or more?
* 1M

* Filtering movies with budget > 25 Cr.
* Grouping by month for all the years and finding movie counts

In [51]:
df_new = df[df['Budget']>=25]

df_new.groupby(['Month'])['Movie_ID'].count().sort_values(ascending=False).reset_index().iloc[0]

Month       February
Movie_ID           9
Name: 0, dtype: object

# OR

* Grouping by each year and it's respective month and finding movie counts

In [50]:
df_new.groupby(['Year','Month'])['Movie_ID'].count().sort_values(ascending=False).reset_index().iloc[0]

Year        2013
Month       July
Movie_ID       5
Name: 0, dtype: object

#### Q6. Which are the top 10 movies with maximum return of investment (ROI)? Calculate ROI as (Box office collection – Budget) / Budget.
* 1M

* Create New Column for Return on Investment(ROI)

In [42]:
df['ROI'] = ((df['BoxOfficeCollection'])-(df['Budget']))/(df['Budget'])

* Choose the top 10 rows with largest ROI

In [43]:
df_new = df.nlargest(10, 'ROI')
print(df_new[['Year','Movie_Name','ROI']].head(10))

     Year                 Movie_Name        ROI
3    2013           Rajdhani Express  53.714286
64   2013                 Aashiqui 2   8.166667
89   2014                         PK   7.647059
132  2013                Grand Masti   7.514286
135  2013               The Lunchbox   7.500000
87   2013                     Fukrey   6.240000
58   2014                   Mary Kom   5.933333
128  2013                     Shahid   5.666667
37   2014  Humpty Sharma Ki Dulhania   5.500000
101  2013         Bhaag Milkha Bhaag   4.466667
